In [2]:
import urllib.request as ur
import requests, re, time
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import pandas as pd
import numpy as np

df_new=pd.read_csv('D:/jupyterNotebook/data.csv')
df_new

,Unnamed: 0,name,type,day,gongmo_p,sicho_p
0,0,쏘카(유가),kospi,20220818,-,-
1,1,에이치와이티씨,kosdaq,20220809,15000,-
2,2,유안타스팩9호,kosdaq,20220804,2000,-
3,3,새빗켐,kosdaq,20220804,35000,-
4,4,수산인더스트리(유가),kospi,20220801,35000,33000
...,...,...,...,...,...,...
1288,1288,맥스엔지니어링,kosdaq,20060704,2900,4285
1289,1289,사이버패스,kosdaq,20060704,5200,4860
1290,1290,인포뱅크,kosdaq,20060704,4600,4610
1291,1291,켐트로닉스,kosdaq,20060117,9500,9800


In [3]:
df_new.drop(['Unnamed: 0'], axis = 1,inplace = True)

In [4]:
df_new

,name,type,day,gongmo_p,sicho_p
0,쏘카(유가),kospi,20220818,-,-
1,에이치와이티씨,kosdaq,20220809,15000,-
2,유안타스팩9호,kosdaq,20220804,2000,-
3,새빗켐,kosdaq,20220804,35000,-
4,수산인더스트리(유가),kospi,20220801,35000,33000
...,...,...,...,...,...
1288,맥스엔지니어링,kosdaq,20060704,2900,4285
1289,사이버패스,kosdaq,20060704,5200,4860
1290,인포뱅크,kosdaq,20060704,4600,4610
1291,켐트로닉스,kosdaq,20060117,9500,9800


In [5]:
df_new = df_new[~df_new['name'].str.contains('스팩')] 
df_new = df_new[~df_new['name'].str.contains('리츠')]

In [6]:
index1 = df_new[df_new['gongmo_p'] == '-'].index
df_new = df_new.drop(index1)
df_new

,name,type,day,gongmo_p,sicho_p
1,에이치와이티씨,kosdaq,20220809,15000,-
3,새빗켐,kosdaq,20220804,35000,-
4,수산인더스트리(유가),kospi,20220801,35000,33000
6,아이씨에이치,kosdaq,20220729,34000,30600
7,성일하이텍,kosdaq,20220728,50000,99900
...,...,...,...,...,...
1288,맥스엔지니어링,kosdaq,20060704,2900,4285
1289,사이버패스,kosdaq,20060704,5200,4860
1290,인포뱅크,kosdaq,20060704,4600,4610
1291,켐트로닉스,kosdaq,20060117,9500,9800


In [7]:
index1 = df_new[df_new['sicho_p'] == '-'].index
df_new = df_new.drop(index1)
df_new

,name,type,day,gongmo_p,sicho_p
4,수산인더스트리(유가),kospi,20220801,35000,33000
6,아이씨에이치,kosdaq,20220729,34000,30600
7,성일하이텍,kosdaq,20220728,50000,99900
8,에이프릴바이오,kosdaq,20220728,16000,19500
10,루닛,kosdaq,20220721,30000,30800
...,...,...,...,...,...
1288,맥스엔지니어링,kosdaq,20060704,2900,4285
1289,사이버패스,kosdaq,20060704,5200,4860
1290,인포뱅크,kosdaq,20060704,4600,4610
1291,켐트로닉스,kosdaq,20060117,9500,9800


In [10]:
df_new = df_new.astype({'gongmo_p':'int64'})
df_new = df_new.astype({'sicho_p':'int64'})
df_new.info()
df_new['count']=np.where(df_new['sicho_p']>df_new['gongmo_p'],1,-1)
df_new

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1036 entries, 4 to 1292
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   name      1036 non-null   object        
 1   type      1036 non-null   object        
 2   day       1036 non-null   datetime64[ns]
 3   gongmo_p  1036 non-null   int64         
 4   sicho_p   1036 non-null   int64         
 5   count     1036 non-null   int32         
dtypes: datetime64[ns](1), int32(1), int64(2), object(2)
memory usage: 52.6+ KB


,name,type,day,gongmo_p,sicho_p,count
4,수산인더스트리(유가),kospi,2022-08-01,35000,33000,-1
6,아이씨에이치,kosdaq,2022-07-29,34000,30600,-1
7,성일하이텍,kosdaq,2022-07-28,50000,99900,1
8,에이프릴바이오,kosdaq,2022-07-28,16000,19500,1
10,루닛,kosdaq,2022-07-21,30000,30800,1
...,...,...,...,...,...,...
1288,맥스엔지니어링,kosdaq,2006-07-04,2900,4285,1
1289,사이버패스,kosdaq,2006-07-04,5200,4860,-1
1290,인포뱅크,kosdaq,2006-07-04,4600,4610,1
1291,켐트로닉스,kosdaq,2006-01-17,9500,9800,1


In [11]:
df_new['day'] = pd.to_datetime(df_new['day'], format='%Y%m%d', errors='raise')
df_new   

,name,type,day,gongmo_p,sicho_p,count
4,수산인더스트리(유가),kospi,2022-08-01,35000,33000,-1
6,아이씨에이치,kosdaq,2022-07-29,34000,30600,-1
7,성일하이텍,kosdaq,2022-07-28,50000,99900,1
8,에이프릴바이오,kosdaq,2022-07-28,16000,19500,1
10,루닛,kosdaq,2022-07-21,30000,30800,1
...,...,...,...,...,...,...
1288,맥스엔지니어링,kosdaq,2006-07-04,2900,4285,1
1289,사이버패스,kosdaq,2006-07-04,5200,4860,-1
1290,인포뱅크,kosdaq,2006-07-04,4600,4610,1
1291,켐트로닉스,kosdaq,2006-01-17,9500,9800,1


In [12]:
data_cross=df_new.loc[::-1]
data_cross['quarter']=data_cross['day'].dt.quarter
data_cross['year']=data_cross['day'].dt.year

C:\Users\KHS\AppData\Local\Temp\ipykernel_19500\1740441362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cross['quarter']=data_cross['day'].dt.quarter
C:\Users\KHS\AppData\Local\Temp\ipykernel_19500\1740441362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cross['year']=data_cross['day'].dt.year


In [13]:
count_sum=0
count_def=0
count=0
result=[]
count_Qt=0
A=0
data_cross

,name,type,day,gongmo_p,sicho_p,count,quarter,year
1292,오스코텍,kosdaq,2006-01-17,9000,13000,1,1,2006
1291,켐트로닉스,kosdaq,2006-01-17,9500,9800,1,1,2006
1290,인포뱅크,kosdaq,2006-07-04,4600,4610,1,3,2006
1289,사이버패스,kosdaq,2006-07-04,5200,4860,-1,3,2006
1288,맥스엔지니어링,kosdaq,2006-07-04,2900,4285,1,3,2006
...,...,...,...,...,...,...,...,...
10,루닛,kosdaq,2022-07-21,30000,30800,1,3,2022
8,에이프릴바이오,kosdaq,2022-07-28,16000,19500,1,3,2022
7,성일하이텍,kosdaq,2022-07-28,50000,99900,1,3,2022
6,아이씨에이치,kosdaq,2022-07-29,34000,30600,-1,3,2022


In [14]:
data_cross['총점']=0
data_cross['카운트']=0
data_cross['분기카운트']=0

C:\Users\KHS\AppData\Local\Temp\ipykernel_19500\1571494592.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cross['총점']=0
C:\Users\KHS\AppData\Local\Temp\ipykernel_19500\1571494592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cross['카운트']=0
C:\Users\KHS\AppData\Local\Temp\ipykernel_19500\1571494592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [15]:
data_cross["range"]=range(len(data_cross))

C:\Users\KHS\AppData\Local\Temp\ipykernel_19500\876808387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cross["range"]=range(len(data_cross))


In [16]:
data_cross=data_cross.set_index('range')

In [165]:
for i in range(len(data_cross)):
    
        if data_cross.iloc[i]["quarter"]!=data_cross.iloc[i-1]["quarter"] and i!=0:
            count_Qt+=round((count_sum+count_def)/count,3)
            result.append(count_Qt)
            
            count_sum=0
            count_def=0
            count_Qt=0
            count=0
            
        if data_cross.iloc[i]["count"]==-1:
            count_def=count_def-1
            count+=1
            data_cross.loc[i,"카운트"]=count
        else:
            count_sum=count_sum+1
            count+=1
            data_cross.loc[i,"카운트"]=count
            
        data_cross.loc[i,"총점"]=count_sum+count_def

In [167]:
data_cross

,name,type,day,gongmo_p,sicho_p,count,quarter,year,총점,카운트
range,,,,,,,,,,
0,오스코텍,kosdaq,2006-01-17,9000,13000,1,1,2006,1,1
1,켐트로닉스,kosdaq,2006-01-17,9500,9800,1,1,2006,2,2
2,인포뱅크,kosdaq,2006-07-04,4600,4610,1,3,2006,1,1
3,사이버패스,kosdaq,2006-07-04,5200,4860,-1,3,2006,0,2
4,맥스엔지니어링,kosdaq,2006-07-04,2900,4285,1,3,2006,1,3
...,...,...,...,...,...,...,...,...,...,...
1031,루닛,kosdaq,2022-07-21,30000,30800,1,3,2022,3,5
1032,에이프릴바이오,kosdaq,2022-07-28,16000,19500,1,3,2022,4,6
1033,성일하이텍,kosdaq,2022-07-28,50000,99900,1,3,2022,5,7


In [169]:
data_cross['Quater_per']=0

In [170]:
result

[1.0,
 0.375,
 1.0,
 1.0,
 1.0,
 0.765,
 0.029,
 -0.25,
 0.5,
 0.167,
 0.5,
 0.667,
 0.917,
 0.455,
 0.52,
 0.545,
 0.467,
 0.2,
 0.455,
 0.176,
 0.176,
 1.0,
 0.385,
 1.0,
 0.0,
 0.111,
 0.111,
 0.5,
 1.0,
 1.0,
 0.429,
 1.0,
 1.0,
 0.4,
 0.484,
 0.333,
 0.692,
 0.333,
 0.282,
 0.8,
 0.6,
 0.467,
 0.091,
 0.5,
 0.333,
 0.217,
 0.333,
 0.714,
 1.0,
 0.579,
 0.263,
 1.0,
 0.333,
 0.273,
 0.455,
 0.5,
 1.0,
 0.655,
 0.793,
 0.75,
 0.375,
 0.6,
 0.583,
 0.3,
 0.6]

In [179]:
x=-10.30.30.30.30.3

In [180]:
for i in range(len(data_cross)):
    if data_cross.iloc[i]["quarter"]==1 and data_cross.iloc[i]["year"]==2006:
        data_cross.loc[i,"Quater_per"]=0
        continue
    elif data_cross.iloc[i]["quarter"]!=data_cross.iloc[i-1]["quarter"]and i!=0:
        print(x)
        x=x+1
        
    data_cross.loc[i,"Quater_per"]=result[x]

-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [173]:
data_cross

,name,type,day,gongmo_p,sicho_p,count,quarter,year,총점,카운트,Quater_per
range,,,,,,,,,,,
0,오스코텍,kosdaq,2006-01-17,9000,13000,1,1,2006,1,1,0.0
1,켐트로닉스,kosdaq,2006-01-17,9500,9800,1,1,2006,2,2,0.0
2,인포뱅크,kosdaq,2006-07-04,4600,4610,1,3,2006,1,1,1.0
3,사이버패스,kosdaq,2006-07-04,5200,4860,-1,3,2006,0,2,1.0
4,맥스엔지니어링,kosdaq,2006-07-04,2900,4285,1,3,2006,1,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1031,루닛,kosdaq,2022-07-21,30000,30800,1,3,2022,3,5,0.6
1032,에이프릴바이오,kosdaq,2022-07-28,16000,19500,1,3,2022,4,6,0.6
1033,성일하이텍,kosdaq,2022-07-28,50000,99900,1,3,2022,5,7,0.6


In [175]:
data_cross.to_csv('C:/Users/KHS/Desktop/Result_per.csv',encoding="CP949")


In [176]:
data_final=data_cross.loc[::-1]

In [177]:
data_final

,name,type,day,gongmo_p,sicho_p,count,quarter,year,총점,카운트,Quater_per
range,,,,,,,,,,,
1035,수산인더스트리(유가),kospi,2022-08-01,35000,33000,-1,3,2022,3,9,0.6
1034,아이씨에이치,kosdaq,2022-07-29,34000,30600,-1,3,2022,4,8,0.6
1033,성일하이텍,kosdaq,2022-07-28,50000,99900,1,3,2022,5,7,0.6
1032,에이프릴바이오,kosdaq,2022-07-28,16000,19500,1,3,2022,4,6,0.6
1031,루닛,kosdaq,2022-07-21,30000,30800,1,3,2022,3,5,0.6
...,...,...,...,...,...,...,...,...,...,...,...
4,맥스엔지니어링,kosdaq,2006-07-04,2900,4285,1,3,2006,1,3,1.0
3,사이버패스,kosdaq,2006-07-04,5200,4860,-1,3,2006,0,2,1.0
2,인포뱅크,kosdaq,2006-07-04,4600,4610,1,3,2006,1,1,1.0


In [181]:
len(result)

65